In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import string
import numpy as np
import pandas as pd

## Pulling all the reviews for a specific game

In [ ]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
headers = {'User-Agent':user_agent}
justcause4_rev = requests.get("https://www.metacritic.com/game/playstation-4/god-of-war/critic-reviews",
                         headers = headers)

In [ ]:
justcause4_details = requests.get("https://www.metacritic.com/game/playstation-4/god-of-war/details",
                         headers = headers)
jc4_detail_soup = BeautifulSoup(justcause4_details.content, 'html.parser')
jc4_detail_soup

In [ ]:
# Getting franchise
def GetFranchise(detail_soup):
    franchise_js = detail_soup.find('script', {"type":"text/javascript",
                                                    "src":"https://urs.metacritic.com/sdk/urs.js"}).previous_sibling.previous_sibling
    franchise_text = franchise_js.get_text().strip().split(';')
    for element in franchise_text:
        element = element.strip()
    #     print(element,'\n')
        if re.match(re.compile('MetaC.Video.setIMATargeting\(\"franchise*'), element):
            match = element
            match = match.split(',')[-1].strip().strip('\)').strip('\"')
            match = match.replace('-', ' ')
            return(match)

GetFranchise(jc4_detail_soup)

In [ ]:
jc4_rev_soup = BeautifulSoup(justcause4_rev.content, 'html.parser')
#jc4_reviews = jc4_soup.find_all('a', text='Read full review')

In [ ]:
jc4_review_metas = jc4_rev_soup.find_all('li', {"class":"review critic_review"})
review_score_dict = {}
review_summary_dict = {}

for rev in jc4_review_metas:
    try:
        cur_rev_critic = rev.find('div', {"class":"review_critic"}).find('a').get_text().strip()
    except:
        cur_rev_critic = rev.find('div', {"class":"review_critic"}).find('div').get_text().strip()
    cur_rev_summary = rev.find('div', {"class":"review_body"}).get_text().strip()
    cur_rev_score = rev.find('div', {"class":"review_grade"}).get_text().strip()
    review_summary_dict[cur_rev_critic] = cur_rev_summary
    review_score_dict[cur_rev_critic] = cur_rev_score

review_summary_dict


In [ ]:
jc4_review_links = [a.get('href') for a in jc4_rev_soup.find_all('a', href=True, text='Read full review')]
jc4_review_links

In [ ]:
just_cause4 = requests.get("https://www.metacritic.com/game/playstation-4/just-cause-4/",
                         headers = headers)
jc4_soup = BeautifulSoup(just_cause4.content, 'html.parser')

In [ ]:
jc4_pub = jc4_detail_soup.find_all('a', {'href': re.compile(r'/company')})[1].get_text().strip()
jc4_details = jc4_detail_soup.find_all('th', scope='row')
jc4_details[1].get_text().strip(':')
jc4_deets = {}
for detail in jc4_details:
    if detail.get_text() == "ESRB Descriptors:":
        print('ESRB')
        value = detail.next_sibling.get_text().strip()
    elif detail.get_text() == "Number of Online Players:":
        value = detail.next_sibling.get_text().strip()
    elif detail.get_text() == "Genre(s):":
        value = detail.next_sibling.next_sibling.get_text().split(',')
        for i in range(len(value)):
            value[i] = value[i].strip()
    else:
        if detail.next_sibling.next_sibling == None:
            print(detail)
        value = detail.next_sibling.next_sibling.get_text().strip()
    jc4_deets[detail.get_text().strip(':')] = value
    
print(jc4_deets)

In [ ]:
page = requests.get("https://www.gamerevolution.com/review/467609-just-cause-4-review-a-cause-for-celebration")
page.status_code == 200

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
body = soup.find('body')
the_contents_of_body_without_body_tags = body.find_all('p')
string = ""
for i in the_contents_of_body_without_body_tags:
    string += i.get_text()
    string += " "
    
print(string)

In [ ]:
all_review_text = {}

for link in jc4_review_links:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    body = soup.find('body')
    body_content = body.find_all('p')
    review_string = ""
    for i in body_content:
        review_string += i.get_text()
        review_string += " "
    all_review_text[link] = review_string
    

In [ ]:
# Making the review text into a list of words
all_reviews_string = "\n".join(all_review_text.values())
all_review_words = all_reviews_string.split()
review_words = []
for word in all_review_words:
    cur_word = word.rstrip(' .,')
    review_words.append(cur_word)
review_words

In [ ]:
filtered_review_words = []
for word in review_words:
    if word in list_of_adjectives:
        filtered_review_words.append(word)
jc4_adjectives = ' '.join(filtered_review_words)
jc4_adjectives = jc4_adjectives.split(' ')

In [ ]:
np.savetxt("Data/jc4.txt", jc4_adjectives, delimiter=",", fmt='%s')

# Doing the above for all games

In [ ]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
headers = {'User-Agent':user_agent}

In [ ]:
# Loading in list of all games
meta_games = pd.read_csv('../Data/result.csv')

meta_games.sort_values(by='metascore',ascending=False).head(20)
#np.unique(meta_games['console'])

# Making a copy of the datafram that I will mutate
meta_games_copy = meta_games.copy()

# Adding new columns for genre, publisher, etc
details = ["genre(s)","developer","players","publisher","rating"] 
extra_details = ["ESRB Descriptors:", "Number of Online Players:", "Special Controllers:", "Number of Players:"]

for new_col in details + extra_details + ['official site']:
    new_col = new_col.strip(':').lower()
    meta_games_copy[new_col] = np.
    
list(meta_games_copy.columns)[5:]

In [ ]:
## MAYBE DONT EVEN NEED TO MERGE, JUST USE READ IN CSV

extra_up_until_now = pd.read_csv("../Data/extra_details_complete.csv", index_col=0).replace('<built-in function empty>', np.empty)
extra_up_until_now['developer'][30]
meta_games_new = pd.merge(meta_games_copy,extra_up_until_now, how='right')
print(len(meta_games_copy),len(meta_games_new))
print(meta_games_copy.head(5),'\n\n')
print(extra_up_until_now.head(5),'\n\n')
print(meta_games_new.head(5),'\n\n')

In [ ]:
def URLMaker(name,platform,end=""):
    base_url = 'https://www.metacritic.com/game'
    platform_dict = {'PS':'playstation',
                    'PS2':'playstation-2',
                    'PS3':'playstation-3',
                    'PS4':'playstation-4',
                    'X360':'xbox-360',
                    'XBOX':'xbox',
                    'PC':'pc',
                    'VITA':'playstation-vita',
                    '3DS':'3ds',
                    'DC':'dreamcast',
                    'DS':'ds',
                    'GBA':'game-boy-advance',
                    'GC':'gamecube',
                    'N64':'nintendo-64',
                    'PSP':'psp',
                    'Switch':'switch',
                    'WII':'wii',
                    'WIIU':'wii-u',
                    'XONE':'xbox-one'}
    full_platform = platform_dict[platform.strip()]
    simplified_name = name.lower().strip()
    # Getting rid of junk from the title
    simplified_name = simplified_name.replace('\'','').replace(',','').replace(':','').replace('/','')
    simplified_name = simplified_name.replace('.','').replace(';','').replace('& ','')
    simplified_name = simplified_name.replace(' ', '-')
    
    complete_url= '/'.join([base_url,full_platform,simplified_name,end])
    return complete_url

urls = []
for row in meta_games.iterrows():
    urls.append(URLMaker(row[1]['name'], row[1]['console']))
urls[-1]
meta_games[1100:1200]

## Getting extra details

In [ ]:
extra_details = ["ESRB Descriptors:", "Number of Online Players:", "Special Controllers:", "Number of Players:"]

for i, row in meta_games.iterrows():
    print(row['name'])
    game_metacritic_url = URLMaker(row['name'], row['console'], 'details')
    game_req = requests.get(game_metacritic_url, headers = headers)
    if game_req.status_code != 200:
        print(game_req.status_code,game_metacritic_url)
    else:
        details_soup = BeautifulSoup(game_req.content, 'html.parser')
        
        # Doing publisher separately, first
        publisher = details_soup.find_all('a', {'href': re.compile(r'/company')})[1].get_text().strip()
        meta_games_copy.at[i,'publisher'] = publisher
        
        
        game_details = details_soup.find_all('th', scope='row')
        for detail in game_details:
            if detail.get_text() in extra_details:
                value = detail.next_sibling.get_text().strip()
            elif detail.get_text() == "Genre(s):":
                value = detail.next_sibling.next_sibling.get_text().split(',')
                for j in range(len(value)):
                    value[j] = value[j].strip()
                value = np.unique(value)
            else:
                value = detail.next_sibling.next_sibling.get_text().strip()
            key = detail.get_text().strip(':').lower()
            meta_games_copy.at[i,key] = value

### Getting the review scores

In [ ]:
for new_col in ['review scores', 'review summaries']:
    meta_games_copy[new_col] = np.empty

stopped = 0

try:
    for i, row in meta_games[stopped:].iterrows():
        print(row['name'], i)
        if i > 10:
            break
        game_metacritic_url = URLMaker(row['name'], row['console'], 'critic-reviews')
        try: 
            game_req = requests.get(game_metacritic_url, headers = headers)
        except:
            meta_games_copy.to_csv('extra_scores_up_until_now.csv')
            print("Request blocked, waiting 15 seconds.")
            time.sleep(15) 
            game_req = requests.get(game_metacritic_url, headers = headers)
        if game_req.status_code != 200:
            print(game_req.status_code,game_metacritic_url)
        else:
            reviews_soup = BeautifulSoup(game_req.content, 'html.parser')
            review_metas = reviews_soup.find_all('li', {"class":"review critic_review"})
            review_score_dict = {}
            review_summary_dict = {}
            for rev in review_metas:
                try:
                    cur_rev_critic = rev.find('div', {"class":"review_critic"}).find('a').get_text().strip()
                except:
                    cur_rev_critic = rev.find('div', {"class":"review_critic"}).find('div').get_text().strip()
                cur_rev_summary = rev.find('div', {"class":"review_body"}).get_text().strip()
                cur_rev_score = rev.find('div', {"class":"review_grade"}).get_text().strip()

                review_summary_dict[cur_rev_critic] = cur_rev_summary
                review_score_dict[cur_rev_critic] = cur_rev_score

                meta_games_copy.at[i,'review scores'] = review_score_dict
                meta_games_copy.at[i, 'review summaries'] = review_summary_dict
except:
    meta_games_copy.to_csv('extra_scores_up_until_now.csv')
            
meta_games_copy.to_csv('extra_scores_complete.csv')
           

In [ ]:
meta_games_copy.head(15)['review summaries'][1]

## Getting list of positive and negative adjectives

In [ ]:
adjectives_4800 = requests.get("https://patternbasedwriting.com/elementary_writing_success/list-4800-adjectives/")
soup_4800 = BeautifulSoup(adjectives_4800.content, 'html.parser')
words_4800 = soup_4800.find_all('table')[2].get_text()
words_4800 = words_4800.split('\n')
words_4800

In [ ]:
list_of_adjectives = []
banned_words = ['the', 'a', 'an', 'its', 'that', 'this', 'my', 'what', 'any', 'their', 'these', 'his', 'her',
                'game', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
for i in range(len(words_4800)):
    if len(words_4800[i]) > 0:
        cur_word = words_4800[i].split()[1]
        if cur_word not in banned_words:
            list_of_adjectives.append(cur_word)

np.savetxt("Data/list_of_adjectives.txt", list_of_adjectives, delimiter=",", fmt='%s')

In [ ]:
# Positive words
with open('words-to-use-pos.txt', 'r') as f:
    w2u_pos = [line.strip() for line in f]

oliver_pos = ['absolutely','accepted','acclaimed','accomplish','accomplishment','achievement','action','active','admire','adorable','adventure','affirmative','affluent','agree','agreeable','amazing','angelic','appealing','approve','aptitude','attractive','awesome','beaming','beautiful','believe','beneficial','bliss','bountiful','bounty','brave','bravo','brilliant','bubbly','calm','celebrated','certain','champ','champion','charming','cheery','choice','classic','classical','clean','commend','composed','congratulation','constant','cool','courageous','creative','cute','dazzling','delight','delightful','distinguished','divine','earnest','easy','ecstatic','effective','effervescent','efficient','effortless','electrifying','elegant','enchanting','encouraging','endorsed','energetic','energized','engaging','enthusiastic','essential','esteemed','ethical','excellent','exciting','exquisite','fabulous','fair','familiar','famous','fantastic','favorable','fetching','fine','fitting','flourishing','fortunate','free','fresh','friendly','fun','funny','generous','genius','genuine','giving','glamorous','glowing','good','gorgeous','graceful','great','green','grin','growing','handsome','happy','harmonious','healing','healthy','hearty','heavenly','honest','honorable','honored','hug','idea','ideal','imaginative','imagine','impressive','independent','innovate','innovative','instant','instantaneous','instinctive','intellectual','intelligent','intuitive','inventive','jovial','joy','jubilant','keen','kind','knowing','knowledgeable','laugh','learned','legendary','light','lively','lovely','lucid','lucky','luminous','marvelous','masterful','meaningful','merit','meritorious','miraculous','motivating','moving','natural','nice','novel','now','nurturing','nutritious','okay','one','open','optimistic','paradise','perfect','phenomenal','pleasant','pleasurable','plentiful','poised','polished','popular','positive','powerful','prepared','pretty','principled','productive','progress','prominent','protected','proud','quality','quick','quiet','ready','reassuring','refined','refreshing','rejoice','reliable','remarkable','resounding','respected','restored','reward','rewarding','right','robust','safe','satisfactory','secure','seemly','simple','skilled','skillful','smile','soulful','sparkling','special','spirited','spiritual','stirring','stunning','stupendous','successsuccessful','sunny','super','superb','supporting','surprising','terrific','thorough','thrilling','thriving','tops','tranquil','transformative','transforming','trusting','truthful','unreal','unwavering','up','upbeat','upright','upstanding','valued','vibrant','victorious','victory','vigorous','virtuous','vital','vivacious','wealthy','welcome','well','whole','wholesome','willing','wonderful','wondrous','worthy','wow','yes','yummy','zeal','zealous']
    
combined_pos = list(np.unique(w2u_pos+oliver_pos))

np.savetxt("Data/positive_adjectives.txt", combined_pos, delimiter=",", fmt='%s')

# Negative words 

with open('words-to-use-neg.txt', 'r') as f:
    w2u_neg = [line.strip() for line in f]
    
oliver_neg = ['abysmal','adverse','alarming','angry','annoy','anxious','apathy','appalling','atrocious','awful','bad','banal','barbed','belligerent','bemoan','beneath','boring','broken','callous','clumsy','coarse','cold','cold','hearted','collapse','confused','contradictory','contrary','corrosive','corrupt','crazy','creepy','criminal','cruel','cry','cutting','damage','damaging','dastardly','dead','decaying','deformed','deny','deplorable','depressed','deprived','despicable','detrimental','dirty','disease','disgusting','disheveled','dishonest','dishonorable','dismal','distress','dreadful','dreary','enraged','eroding','evil','fail','faulty','fear','feeble','fight','filthy','foul','frighten','frightful','gawky','ghastly','grave','greed','grim','grimace','gross','grotesque','gruesome','guilty','haggard','hard','hard','hearted','harmful','hate','hideous','homely','horrendous','horrible','hostile','hurt','hurtful','icky','ignorant','ignore','ill','immature','imperfect','impossible','inane','inelegant','infernal','injure','injurious','insane','insidious','insipid','jealous','junky','lose','lousy','lumpy','malicious','mean','menacing','messy','misshapen','missing','misunderstood','moan','moldy','monstrous','naive','nasty','naughty','negate','negative','never','no','nobody','nondescript','nonsense','not','noxious','objectionable','odious','offensive','old','oppressive','pain','perturb','pessimistic','petty','plain','poisonous','poor','prejudice','questionable','quirky','quit','reject','renege','repellant','reptilian','repugnant','repulsive','revenge','revolting','rocky','rotten','rude','ruthless','sad','savage','scare','scary','scream','severe','shocking','shoddy','sick','sickening','sinister','slimy','smelly','sobbing','sorry','spiteful','sticky','stinky','stormy','stressful','stuck','stupid','substandard','suspect','suspicious','tense','terrible','terrifying','threatening','ugly','undermine','unfair','unfavorable','unhappy','unhealthy','unjust','unlucky','unpleasant','unsatisfactory','unsightly','untoward','unwanted','unwelcome','unwholesome','unwieldy','unwise','upset','vice','vicious','vile','villainous','vindictive','wary','weary','wicked','woeful','worthless','wound','yell','yucky','zero']

combined_neg = np.unique(w2u_neg+oliver_neg)

np.savetxt("Data/negative_adjectives.txt", combined_neg, delimiter=",", fmt='%s')

In [ ]:
# Removing the words not in the adjective list from the review words
filtered_review_words = []
for word in review_words:
    if word in list_of_adjectives:
        filtered_review_words.append(word)
' '.join(filtered_review_words)

### Final scores data wrangling

In [ ]:
scores_complete = pd.read_csv('C:/Users/Admin/Desktop/scores_complete.csv', index_col=0)
original = pd.read_csv('../Data/result.csv')
scores_complete

In [ ]:
o2 = original.join(scores_complete.drop_duplicates(['review summaries'])[['review scores','review summaries' ]])

o2['userscore'] = o2['userscore'].replace('tbd', 'NaN').astype(float)
o2.loc[(o2['console']=='PS3') & (o2['userscore']>8.3)]
#o2.to_csv('scores_and_summaries.csv')

### Final details wrangling
Need to delete players column (redundant to number of players and number of online players columns), remove the word 'Series' from all franchise cells, and properly delimit the 

In [ ]:
import ast
details = pd.read_csv('../Data/extra_details_complete.csv',index_col=0,encoding='utf-8',dtype={'metascore':np.int,'genre(s)':'O'})
details = details.drop(columns=['players'])
pubs = np.unique(details['publisher'], return_counts=True)
N = 10
print('There are', len(pubs[0][pubs[1] > N]),'publishers with more than', N, 'games')
devs = np.unique(details['developer'], return_counts=True)
print('There are', len(devs[0][devs[1] > N]),'developers with more than', N, 'games')

#gens = details['genre(s)'][1].replace(' u\'', ',\'').replace('[u','[')


details['franchise'] = details['franchise'].str.replace(' Series', '').replace(np.nan,'')
details = details.replace('<built-in function empty>', ' ')
details['genre(s)'] = details['genre(s)'].str.replace(' u\'', ', \'').str.replace('u\'','\'').str.replace('\'', '').str.replace('\]','').str.replace('\[','')
#details.to_csv('./extra_details_complete.csv')

### Wrangling user comments

In [ ]:
user_comments = pd.read_csv('../Data/metacritic_game_user_comments.csv', index_col=0, dtype=str)
critic_reviews = pd.read_csv('../Data/scores_and_summaries.csv',index_col=0)
critic_reviews_copy = critic_reviews.copy()
user_comments_copy = user_comments.copy().drop('Username', axis=1)
user_comments_copy.head()

In [ ]:
# I think unnecessary


# import unicodedata
# for i, row in user_comments.iterrows():
#     if type(row['Comment']) != str:
#         print(user_comments_copy.iloc[i]['Comment'])
#         continue
#     else:
#         user_comments_copy.iloc[i]['Comment'] = unicodedata.normalize('NFKD',row['Comment']).encode('ascii', 'ignore').decode('utf-8')

In [ ]:
user_comments_copy.fillna(value = '', inplace=True)
user_comments_copy.isna().values.any()
#user_comments_copy.loc[15:40]

In [ ]:
grouped_comments = user_comments_copy.groupby(['Title', 'Platform']).agg({'Userscore':', '.join, 
                             'Comment': ', '.join}).reset_index()
grouped_comments

In [ ]:
critic_reviews

In [ ]:
comments_consoles = np.unique(user_comments['Platform'])
critics_consoles = np.unique(critic_reviews['console'])
print(comments_consoles)
print(critics_consoles)
gro

In [ ]:
console_convert_dict = {'Dreamcast':'DC',
                        'GameBoyAdvance':'GBA',
                        'GameCube':'GC',
                        'Nintendo64':'N64',
                        'PlayStation':'PS',
                        'PlayStation2':'PS2',
                        'PlayStation3':'PS3',
                        'PlayStation4':'PS4',
                        'PlayStationVita':'VITA',
                        'Wii':'WII',
                        'WiiU':'WIIU',
                        'Xbox':'XBOX',
                        'Xbox360':'X360',
                        'XboxOne':'XONE',
                        'not specified':'NA'}

grouped_comments_copy = grouped_comments.copy()

for i, row in grouped_comments.iterrows():
    platform = row['Platform']
    if platform in console_convert_dict.keys():
        grouped_comments_copy.loc[i]['Platform'] = console_convert_dict[platform]
    else:
        continue
grouped_comments_copy.rename(index=str, columns={"Platform": "console",
                                                 "Title": "name",
                                                 'Userscore': 'user scores',
                                                 'Comment':'user comments'},inplace=True)


In [ ]:
print(critic_reviews_copy.columns)
print(grouped_comments_copy.columns)

In [ ]:
combined = pd.merge(critic_reviews_copy, grouped_comments_copy,on=['name','console'], how='left')
combined

In [ ]:
n = 0
combined_copy = combined.copy()
for i,row in combined.iterrows():
    try:
        combined_copy.at[i,'review scores'] = re.sub(r'[a-zA-Z]+',',',re.sub(r'[\W+]', '',combined.loc[i,'review scores']))[1:]
        combined_copy.at[i,'review summaries'] = ' '.join(list(eval(combined.loc[i,'review summaries']).values()))
    except:
        print(combined_copy.loc[i,'review scores'])
        print(combined_copy.loc[i,'review summaries'])
        combined_copy.at[i,'review scores'] = np.nan
        combined_copy.at[i,'review summaries'] = np.nan
combined

In [ ]:
combined_copy.loc[1,'review summaries']


In [ ]:
#combined_copy.to_csv('../Data/critic_and_users_full_text.csv')

## Filtering for only adjectives

In [ ]:
adj_4800 = np.loadtxt('../Data/list_of_adjectives.txt',delimiter=',',dtype='str')
pos_words = np.loadtxt('../Data/positive_adjectives.txt',delimiter=',',dtype='str')
neg_words = np.loadtxt('../Data/negative_adjectives.txt',delimiter=',',dtype='str')
all_adj = frozenset(list(np.union1d(np.union1d(pos_words,neg_words),adj_4800)))

only_adj_df = combined_copy.copy()
only_adj_df['user adjectives'] = ''
only_adj_df['critic adjectives'] = ''

for i, row in combined_copy.iterrows():
    if i%1000 == 0:
        print(i)
    try:
        critic_words = re.sub(r'[^a-z\s\-]', '',only_adj_df.loc[i,'review summaries'].lower()).split(' ')
        user_adj_gen = [c_word for c_word in critic_words if c_word in all_adj]
        only_adj_df.at[i,'critic adjectives'] = ','.join(list(user_adj_gen))
    except:
        pass
    try:
        user_words = re.sub(r'[^a-z\s\-]', '',only_adj_df.loc[i,'user comments'].lower()).split(' ')
        user_adj_gen = (u_word for u_word in user_words if u_word in all_adj)
        only_adj_df.at[i,'user adjectives'] = ','.join(list(user_adj_gen))
    except:
        continue

In [ ]:
words = re.sub(r'[^a-z\s\-]', '',only_adj_df.loc[0,'review summaries'].lower()).split(' ')
found = [i for i in words if i in all_adj]
print(list(found))

In [ ]:
# only_adj_df = only_adj_df.drop('review summaries', axis=1)
# only_adj_df = only_adj_df.drop('user comments', axis=1)
only_adj_df.head(20)
#only_adj_df.to_csv('../Data/critic_and_users_adjective_only.csv')